In [12]:
import requests
import json
from datetime import datetime, timedelta

# API를 요청할 URL과 인증키
BASE_URL = "https://www.koreaexim.go.kr/site/program/financial/exchangeJSON"
AUTH_KEY = "1p25bUnirmJUSZgqS9iWm4HrFM2ODhra"  # 여기에 본인의 인증키를 넣어주세요

# 오늘 날짜부터 지정한 day 수만큼 날짜를 생성하는 함수
def generate_dates(start_date, total_days):
    dates = []
    end_date = start_date - timedelta(days=total_days)
    while start_date > end_date:
        dates.append(start_date.strftime("%Y%m%d"))
        start_date -= timedelta(days=1)
    return dates

# 주어진 날짜에 대해 USD 환율 정보를 요청하는 함수
def fetch_usd_data_for_date(date):
    params = {
        "authkey": AUTH_KEY,
        "searchdate": date,
        "data": "AP01"
    }
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        return None

# USD 데이터를 날짜별로 저장하는 함수
def get_usd_bkpr_over_days(start_date, total_days):
    dates = generate_dates(start_date, total_days)
    usd_data = {}

    for date in dates:
        data = fetch_usd_data_for_date(date)
        
        # 빈 배열인지 확인 (휴일인 경우)
        if data and isinstance(data, list) and len(data) > 0:
            # USD 데이터를 찾기
            for entry in data:
                if entry.get("cur_unit") == "USD":
                    usd_data[date] = entry.get("bkpr")
                    usd_data[date] = usd_data[date].replace(',', '') # 통화에서 쉽표 제거하기
                    break  # USD 데이터를 찾으면 더 이상 탐색하지 않음
        else:
            # 휴일인 경우
            print(f"Data unavailable for {date} (possibly a holiday)")

    return usd_data

In [13]:
# 시작 날짜를 설정 (2024년 10월 4일)
start_date = datetime(2024, 10, 4)

# 365 * 3일치 USD bkpr 데이터를 가져오기
total_days = 3
usd_bkpr_data = get_usd_bkpr_over_days(start_date, total_days)

# # 결과 출력
print("USD bkpr values over the specified number of days:")
for date, bkpr in usd_bkpr_data.items():
    print(f"{date}: {bkpr}")

Data unavailable for 20241003 (possibly a holiday)
USD bkpr values over the specified number of days:
20241004: 1320
20241002: 1306


In [14]:
import sqlite3
import os

# sqlite3db 디렉토리가 없으면 생성
if not os.path.exists('sqlite3db'):
    os.makedirs('sqlite3db')

# sqlite3db 디렉토리 아래에 데이터베이스 파일 경로 설정
db_path = os.path.join('sqlite3db', 'exchange_rates.db')

# 데이터베이스 파일에 연결 (파일이 없으면 새로 생성됨)
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# 테이블 생성 (존재하지 않으면 생성)
# date를 PRIMARY KEY로 설정
cursor.execute('''
    CREATE TABLE IF NOT EXISTS exchange_rates (
        date TEXT PRIMARY KEY,
        currency TEXT,
        bkpr REAL
    )
''')

In [15]:
# 데이터를 테이블에 삽입하거나 중복 발생 시 업데이트하는 함수
def insert_or_update_exchange_rate_data(data):
    for date, bkpr in data.items():
        # 중복된 date 값이 있을 경우 bkpr 값을 업데이트
        cursor.execute('''
            INSERT INTO exchange_rates (date, currency, bkpr)
            VALUES (?, ?, ?)
            ON CONFLICT(date) DO UPDATE SET bkpr=excluded.bkpr
        ''', (date, 'USD', bkpr))

In [17]:
# 시작 날짜를 설정 (2024년 10월 4일)
start_date = datetime(2024, 10, 4)

# get_usd_bkpr_over_days 함수로 365 * 3일에 해당하는 환율 정보 가져오기
usd_bkpr_data = get_usd_bkpr_over_days(start_date, 30)

# 테이블에 데이터 삽입 또는 업데이트
insert_or_update_exchange_rate_data(usd_bkpr_data)

# 변경사항 저장
conn.commit()

# 데이터 확인을 위해 조회
cursor.execute('SELECT * FROM exchange_rates')
rows = cursor.fetchall()

# 결과 출력
for row in rows:
    print(row)

# 연결 종료
conn.close()

Data unavailable for 20241003 (possibly a holiday)
Data unavailable for 20241001 (possibly a holiday)
Data unavailable for 20240929 (possibly a holiday)
Data unavailable for 20240928 (possibly a holiday)
Data unavailable for 20240922 (possibly a holiday)
Data unavailable for 20240921 (possibly a holiday)
Data unavailable for 20240918 (possibly a holiday)
Data unavailable for 20240917 (possibly a holiday)
Data unavailable for 20240916 (possibly a holiday)
Data unavailable for 20240915 (possibly a holiday)
Data unavailable for 20240914 (possibly a holiday)
Data unavailable for 20240908 (possibly a holiday)
Data unavailable for 20240907 (possibly a holiday)
('20241004', 'USD', 1320.0)
('20241002', 'USD', 1306.0)
('20240930', 'USD', 1319.0)
('20240927', 'USD', 1329.0)
('20240926', 'USD', 1328.0)
('20240925', 'USD', 1334.0)
('20240924', 'USD', 1335.0)
('20240923', 'USD', 1329.0)
('20240920', 'USD', 1331.0)
('20240919', 'USD', 1331.0)
('20240913', 'USD', 1339.0)
('20240912', 'USD', 1340.0)
(